In [1]:
import pandas as pd
import numpy as np

In [4]:
loc_training = r"C:\Users\FacundoTorraca\Documents\Facultad\Organizacion de Datos\Trabajo Practico 2\Sets de Entrenamiento"

In [68]:
dtypes_ins = { "application_id": np.int16, "ref_type":np.float64, "device_model":np.float64,
               "device_countrycode":np.float64, "device_brand": np.float64, "session_user_agent":"category",
               "kind":"category", "wifi":"category", "device_language":np.float64}   

In [69]:
ins_18_20 = pd.read_csv( loc_training + "\\ins_1.csv", dtype = dtypes_ins )
ins_18_20 = ins_18_20.drop( ["created", "attributed", "implicit", "session_user_agent", "event_uuid", "kind", "wifi", "user_agent", "ref_hash", "click_hash", "trans_id"], axis = 1 )

In [78]:
ins_21_23_tgt = pd.read_csv( loc_training + "\\ins_4.csv", dtype = dtypes_ins )

ins_21_23_tgt = ins_21_23_tgt.drop(  ["created", "attributed", "implicit", "session_user_agent", "event_uuid", "kind", "wifi", "user_agent", "click_hash", "trans_id"], axis = 1 )

ins_21_23_ans = ins_21_23_tgt[ ["ref_hash", "_sc"] ]
del( ins_21_23_tgt["_sc"] ); del( ins_21_23_tgt["ref_hash"] );

### XGBoost

In [79]:
y_train = ins_18_20[ ["_sc"] ]; x_train = ins_18_20.iloc[ :,:-1 ] 

In [80]:
x_test = ins_21_23_tgt; y_test = ins_21_23_ans[["_sc"]]

In [81]:
import xgboost as xgb

In [82]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)

In [83]:
xg_reg.fit(x_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [84]:
preds = xg_reg.predict(x_test)

In [87]:
pd.Series( preds ).head()

0    82572.296875
1    82572.296875
2    82572.296875
3    82572.296875
4    82480.500000
dtype: float32

In [88]:
y_test.head()

,_sc
0,227792.913
1,238454.756
2,196360.906
3,184358.142
4,236046.764


In [90]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 89826.642240
